In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
from Utils.Trainer3DAug16 import *
from Models.LargePNet3D import *

In [4]:
head_dir = r'F:\Datasets\Debackground'
loaded_data = np.load(head_dir + r'\Data.npz')
X_train=loaded_data['X_train']  
y_train=loaded_data['y_train']
X_val=loaded_data['X_val']
y_val= loaded_data['y_val']
X_train= np.transpose(X_train, (0,2,3,1))
y_train=np.transpose(y_train, (0,2,3,1))
X_val= np.transpose(X_val, (0,2,3,1))
y_val= np.transpose(y_val, (0,2,3,1))
print(X_train.shape)
log_dir = head_dir + r'\logfile'
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

(22, 1024, 1024, 24)


In [5]:
img_num = X_val.shape[0]
wanted_num = 50
X_val_expand = []
y_val_expand = []
count = 0
for i in tqdm(range(img_num), desc="Processing images"):  
    raw = X_val[i,...]
    gt = y_val[i,...]
    for j in range (wanted_num//img_num):
        Augraw, Auggt = DataAugZ(raw,gt,512,16)

        X_val_expand.append(Augraw)
        y_val_expand.append(Auggt)
        count = count + 1
X_val_expand = np.array(X_val_expand)
y_val_expand = np.array(y_val_expand)
X_val_expand = np.transpose(X_val_expand, (0,3,1,2))
y_val_expand = np.transpose(y_val_expand, (0,3,1,2))
print(X_val_expand.shape)

Processing images: 100%|██████████| 3/3 [00:04<00:00,  1.52s/it]


(48, 16, 512, 512)


In [6]:
X_train = torch.tensor(X_train, dtype=torch.float16)
y_train = torch.tensor(y_train, dtype=torch.float16)
X_val = torch.tensor(X_val_expand, dtype=torch.float16).unsqueeze(1)
y_val = torch.tensor(y_val_expand, dtype=torch.float16).unsqueeze(1)


train_data = Data.TensorDataset(X_train,y_train)
val_data = Data.TensorDataset(X_val,y_val)

print("X_train.shape",X_train.shape)
print("y_train.shape",y_train.shape)
print("X_val.shape",X_val.shape)
print("y_val.shape",y_val.shape)

X_train.shape torch.Size([22, 1024, 1024, 24])
y_train.shape torch.Size([22, 1024, 1024, 24])
X_val.shape torch.Size([48, 1, 16, 512, 512])
y_val.shape torch.Size([48, 1, 16, 512, 512])


In [7]:
# For MINet
class Options:
    def __init__(self):
        self.LR = 0.001
        self.batchsize = 1
        self.epoch_num = 300
        self.epoch_critic = 10
        self.useinit = 0
        self.MAE_weight = 0.05
        self.SSIM_weight = 0  
        self.StepLR = 0
        self.SteLR = 0
        self.ExpoLR = 0
        self.minibatchnum = 50
        self.patchsize = 512
        self.patchsizez = 16
        self.img_num = X_train.shape[0]
        self.Epochcritic_Point = [50,200]
        self.Epochcritic_Decay = [2,10]
        self.val = 1
opt = Options()

In [8]:
print(y_val.shape)

torch.Size([48, 1, 16, 512, 512])


In [9]:
print(y_val.max())

tensor(1., dtype=torch.float16)


In [ ]:
model = LargePNet3D(1,1,15,4)
save_path = head_dir +r'\logfile\MINet'
model = TrainerAug(model, train_data, val_data, save_path, opt )